### 优化器

In [19]:
import torch
import torch.nn.functional as F
from torch import nn

#### 参数组 (param_groups)

In [3]:
import torch.optim as optim


w1 = torch.randn(2, 2)
w1.requires_grad = True

w2 = torch.randn(2, 2)
w2.requires_grad = True

w3 = torch.randn(2, 2)
w3.requires_grad = True

# 一个参数组
optimizer_1 = optim.SGD([w1, w3], lr=0.1)
print('len(optimizer.param_groups): ', len(optimizer_1.param_groups))
print(optimizer_1.param_groups, '\n')

# 两个参数组
'''
参数组在 optimizer 中表现为一个 list(self.param_groups)，其中每个元素是dict，表示一个参数及其相应配置，
在 dict 中包含'params'、'weight_decay'、'lr' 、'momentum'等字段。
'''
optimizer_2 = optim.SGD([{'params': w1, 'lr': 0.1},
                         {'params': w2, 'lr': 0.001}])
print('len(optimizer.param_groups): ', len(optimizer_2.param_groups))
print(optimizer_2.param_groups)

len(optimizer.param_groups):  1
[{'params': [tensor([[-1.0558, -1.6159],
        [ 0.8573, -2.3439]], requires_grad=True), tensor([[ 0.4339, -0.9652],
        [ 0.2432,  1.8470]], requires_grad=True)], 'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}] 

len(optimizer.param_groups):  2
[{'params': [tensor([[-1.0558, -1.6159],
        [ 0.8573, -2.3439]], requires_grad=True)], 'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}, {'params': [tensor([[ 1.4977,  0.5828],
        [ 0.6707, -0.2233]], requires_grad=True)], 'lr': 0.001, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}]


In [16]:
optimizer_2.param_groups

[{'params': [tensor([[-1.0558, -1.6159],
           [ 0.8573, -2.3439]], requires_grad=True)],
  'lr': 0.1,
  'momentum': 0,
  'dampening': 0,
  'weight_decay': 0,
  'nesterov': False},
 {'params': [tensor([[ 1.4977,  0.5828],
           [ 0.6707, -0.2233]], requires_grad=True)],
  'lr': 0.001,
  'momentum': 0,
  'dampening': 0,
  'weight_decay': 0,
  'nesterov': False}]

### 梯度清零

In [4]:
# ----------------------------------- zero_grad

w1 = torch.randn(2, 2)
w1.requires_grad = True

w2 = torch.randn(2, 2)
w2.requires_grad = True

optimizer = optim.SGD([w1, w2], lr=0.001, momentum=0.9)

optimizer.param_groups[0]['params'][0].grad = torch.randn(2, 2)

print('参数w1的梯度：')
print(optimizer.param_groups[0]['params'][0].grad, '\n')  # 参数组，第一个参数(w1)的梯度

optimizer.zero_grad()
print('执行zero_grad()之后，参数w1的梯度：')
print(optimizer.param_groups[0]['params'][0].grad)  # 参数组，第一个参数(w1)的梯度

参数w1的梯度：
tensor([[ 0.6420,  0.3585],
        [ 1.7008, -0.1880]]) 

执行zero_grad()之后，参数w1的梯度：
tensor([[0., 0.],
        [0., 0.]])


In [15]:
optimizer.param_groups

[{'params': [tensor([[-0.1268, -0.5123],
           [ 0.5882,  0.0946]], requires_grad=True),
   tensor([[-0.1072,  1.1358],
           [-0.1049,  1.3049]], requires_grad=True)],
  'lr': 0.001,
  'momentum': 0.9,
  'dampening': 0,
  'weight_decay': 0,
  'nesterov': False}]

### 获取模型当前的参数，以一个有序字典形式返回

In [20]:
# ----------------------------------- state_dict
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1 * 3 * 3, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 1 * 3 * 3)
        x = F.relu(self.fc1(x))
        return x


net = Net()

# 获取网络当前参数
net_state_dict = net.state_dict()

print('net_state_dict类型：', type(net_state_dict))
print('net_state_dict管理的参数: ', net_state_dict.keys())
for key, value in net_state_dict.items():
    print('参数名: ', key, '\t大小: ', value.shape)

net_state_dict类型： <class 'collections.OrderedDict'>
net_state_dict管理的参数:  odict_keys(['conv1.weight', 'conv1.bias', 'fc1.weight', 'fc1.bias'])
参数名:  conv1.weight 	大小:  torch.Size([1, 3, 3, 3])
参数名:  conv1.bias 	大小:  torch.Size([1])
参数名:  fc1.weight 	大小:  torch.Size([2, 9])
参数名:  fc1.bias 	大小:  torch.Size([2])


### 将 state_dict 中的参数加载到当前网络

In [21]:
# ----------------------------------- load_state_dict

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(1 * 3 * 3, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = x.view(-1, 1 * 3 * 3)
        x = F.relu(self.fc1(x))
        return x

    def zero_param(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                torch.nn.init.constant_(m.weight.data, 0)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                torch.nn.init.constant_(m.weight.data, 0)
                m.bias.data.zero_()
net = Net()

# 保存，并加载模型参数(仅保存模型参数)
torch.save(net.state_dict(), 'net_params.pkl')   # 假设训练好了一个模型net
pretrained_dict = torch.load('net_params.pkl')

# 将net的参数全部置0，方便对比
net.zero_param()
net_state_dict = net.state_dict()
print('conv1层的权值为:\n', net_state_dict['conv1.weight'], '\n')

# 通过load_state_dict 加载参数
net.load_state_dict(pretrained_dict)
print('加载之后，conv1层的权值变为:\n', net_state_dict['conv1.weight'])

conv1层的权值为:
 tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]]) 

加载之后，conv1层的权值变为:
 tensor([[[[-0.0696,  0.0299,  0.0624],
          [-0.1281, -0.1762,  0.0200],
          [ 0.0929, -0.0608, -0.1759]],

         [[-0.0801, -0.1615, -0.1500],
          [ 0.0214, -0.1653, -0.1576],
          [-0.1875, -0.1405, -0.1172]],

         [[ 0.1052, -0.1635,  0.1350],
          [-0.0646, -0.0185,  0.1848],
          [-0.1506, -0.0744,  0.0389]]]])


### 给 optimizer 管理的参数组中增加一组参数，可为该组参数 定制  lr,momentum, 

In [22]:
# ----------------------------------- add_param_group

w1 = torch.randn(2, 2)
w1.requires_grad = True

w2 = torch.randn(2, 2)
w2.requires_grad = True

w3 = torch.randn(2, 2)
w3.requires_grad = True

# 一个参数组
optimizer_1 = optim.SGD([w1, w2], lr=0.1)
print('当前参数组个数: ', len(optimizer_1.param_groups))
print(optimizer_1.param_groups, '\n')

# 增加一个参数组
print('增加一组参数 w3\n')
optimizer_1.add_param_group({'params': w3, 'lr': 0.001, 'momentum': 0.8})

print('当前参数组个数: ', len(optimizer_1.param_groups))
print(optimizer_1.param_groups, '\n')

print('可以看到，参数组是一个list，一个元素是一个dict，每个dict中都有lr, momentum等参数，这些都是可单独管理，单独设定，十分灵活！')

当前参数组个数:  1
[{'params': [tensor([[0.4280, 1.7579],
        [0.7520, 0.0159]], requires_grad=True), tensor([[-0.5168,  0.2112],
        [-1.0788, -0.7243]], requires_grad=True)], 'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}] 

增加一组参数 w3

当前参数组个数:  2
[{'params': [tensor([[0.4280, 1.7579],
        [0.7520, 0.0159]], requires_grad=True), tensor([[-0.5168,  0.2112],
        [-1.0788, -0.7243]], requires_grad=True)], 'lr': 0.1, 'momentum': 0, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}, {'params': [tensor([[-0.8405,  0.6584],
        [ 1.0136, -0.9179]], requires_grad=True)], 'lr': 0.001, 'momentum': 0.8, 'dampening': 0, 'weight_decay': 0, 'nesterov': False}] 

可以看到，参数组是一个list，一个元素是一个dict，每个dict中都有lr, momentum等参数，这些都是可单独管理，单独设定，十分灵活！
